In [9]:
from typing import List


def compute_probability(time: int, time_distribution: List[int]) -> float:
    """
    Compute the probability of a train departing or arriving at a given time
    given a list of departure times or arrival times
    """
    # count the number of trains departing/arriving at the given time
    count = time_distribution.count(time)
    # count the total number of trains
    total = len(time_distribution)
    # compute the probability
    probability = count / total
    return probability


def compute_probability_to_arrive_at_or_before(
    time_limit: int, arrival_times: List[int]
) -> float:
    """
    Compute the probability of arriving at or before a given time
    given a list of arrival times
    """
    # get first (minimum) arrival time
    min_arrival_time = arrival_times[
        0
    ]  # note: should be the first, we assume we get a sorted list
    # get last (maximum) arrival time -> either the latest arrival time or the time limit if that is earlier
    max_arrival_time = min(arrival_times[-1], time_limit)
    # for each arrival time, compute the probability of arriving at that time (which is the probability of arriving at or before the time limit)
    probabilities = [
        compute_probability(time, arrival_times)
        for time in range(min_arrival_time, max_arrival_time + 1)
    ]
    # sum the probabilities
    probability = sum(probabilities)
    return probability

In [2]:
# @todo -> this is a simplification, we need to find a data structure that allows us to store the stations and their connections
# @todo -> here we only have one "destination" per (unique) station, but in reality, we can have multiple connections -> but only one for a unique trip!
trips_graph = {
    # departure node (=Liestal) -> [(scheduled departure_time, neighbor station, scheduled arrival_time, trip identifier)]
    # @todo -> add reliability of the connection
    # departure station, then as list the array of connections with the planned departure time, the neighbor station, the planned arrival time, the trip identifier and the tuple of actual departure and arrival times
    "Zurich": [(5, "Olten", 15, "IC6", [(5, 15), (6, 16), (10, 20)])],
    "Olten": [(20, "Bern", 30, "IC6", [(20, 30), (20, 32), (25, 35)])],
    "Bern": [(35, "Brig", 45, "IC6", [(35, 45), (37, 47), (40, 50)])],
    "Brig": [
        (55, "Milan", 65, "IC6", [(55, 65), (57, 67), (60, 70)])
    ],  #'Milan' not important for this example
}


stations = {
    "Zurich": {
        # @todo use different structure -> tuples of arrival and departure times
        "arr_dep_tuples": [(0, 5), (0, 6), (0, 10)],
        "arrival_times": [0, 0, 0],
        "departure_times": [5, 6, 10],
        "to": "Olten",
    },
    "Olten": {
        "arr_dep_tuples": [(15, 20), (15, 20), (20, 25)],
        "arrival_times": [15, 15, 20],
        "departure_times": [20, 20, 25],
        "to": "Bern",
    },
    "Bern": {
        "arr_dep_tuples": [(30, 35), (32, 37), (35, 40)],
        "arrival_times": [30, 32, 35],
        "departure_times": [35, 37, 40],
        "to": "Brig",
    },
    "Brig": {
        "arr_dep_tuples": [(45, 50), (47, 52), (50, 55)],
        "arrival_times": [45, 47, 50],
        "departure_times": [55, 57, 60],  # not important for this example
        "to": "Milan",  # not important for this example
    },
}

In [3]:
# first, create unique lists of the departure and arrival times for each station
# @todo -> question: we should not sort it, because the order means which departure and arrival times belong together?
# @todo -> a unique list here is also wrong -> otherwise the probabilities are wrong
# for station in stations.values():
#     station['arrival_times'] = list(set(station['arrival_times']))
#     station['departure_times'] = list(set(station['departure_times']))


# add the departure probabilities -> creating a tuple from the departure time and the probability
for station in stations.values():
    station["departure_probabilities"] = [
        (time, compute_probability(time, station["departure_times"]))
        for time in station["departure_times"]
    ]

stations

{'Zurich': {'arr_dep_tuples': [(0, 5), (0, 6), (0, 10)],
  'arrival_times': [0, 0, 0],
  'departure_times': [5, 6, 10],
  'to': 'Olten',
  'departure_probabilities': [(5, 0.3333333333333333),
   (6, 0.3333333333333333),
   (10, 0.3333333333333333)]},
 'Olten': {'arr_dep_tuples': [(15, 20), (15, 20), (20, 25)],
  'arrival_times': [15, 15, 20],
  'departure_times': [20, 20, 25],
  'to': 'Bern',
  'departure_probabilities': [(20, 0.6666666666666666),
   (20, 0.6666666666666666),
   (25, 0.3333333333333333)]},
 'Bern': {'arr_dep_tuples': [(30, 35), (32, 37), (35, 40)],
  'arrival_times': [30, 32, 35],
  'departure_times': [35, 37, 40],
  'to': 'Brig',
  'departure_probabilities': [(35, 0.3333333333333333),
   (37, 0.3333333333333333),
   (40, 0.3333333333333333)]},
 'Brig': {'arr_dep_tuples': [(45, 50), (47, 52), (50, 55)],
  'arrival_times': [45, 47, 50],
  'departure_times': [55, 57, 60],
  'to': 'Milan',
  'departure_probabilities': [(55, 0.3333333333333333),
   (57, 0.3333333333333333)

In [10]:
# for simplicity -> first step, compute probability that we arrive at Olten with a start time 0, budget of 17 and transfer time of 5 minutes (as the default transfer time)
# this is only an intermediate step to understand the reliability computation better

# @todo -> this is a simplification, because I know that I go from Zurich to Olten, but in reality, I need to find the connection that goes from Zurich to Olten
compute_probability_to_arrive_at_or_before(15, stations["Olten"]["arrival_times"])
compute_probability_to_arrive_at_or_before(20, stations["Olten"]["arrival_times"])

# add the arrival probabilities (in the same order as the arrival times) to the stations
for station in stations.values():
    # @todo / note: this only is correct between the first and second station, because we need to consider making the other connections for further stations
    station["arrival_probabilities"] = [
        (
            time,
            compute_probability_to_arrive_at_or_before(time, station["arrival_times"]),
        )
        for time in station["arrival_times"]
    ]
stations

time limit: 15
arrival times: [15, 15, 20]
max arrival time: 15
probabilities: [0.6666666666666666]
time limit: 20
arrival times: [15, 15, 20]
max arrival time: 20
probabilities: [0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333]
time limit: 0
arrival times: [0, 0, 0]
max arrival time: 0
probabilities: [1.0]
time limit: 0
arrival times: [0, 0, 0]
max arrival time: 0
probabilities: [1.0]
time limit: 0
arrival times: [0, 0, 0]
max arrival time: 0
probabilities: [1.0]
time limit: 15
arrival times: [15, 15, 20]
max arrival time: 15
probabilities: [0.6666666666666666]
time limit: 15
arrival times: [15, 15, 20]
max arrival time: 15
probabilities: [0.6666666666666666]
time limit: 20
arrival times: [15, 15, 20]
max arrival time: 20
probabilities: [0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333]
time limit: 30
arrival times: [30, 32, 35]
max arrival time: 30
probabilities: [0.3333333333333333]
time limit: 32
arrival times: [30, 32, 35]
max arrival time: 32
probabilities: [

{'Zurich': {'arr_dep_tuples': [(0, 5), (0, 6), (0, 10)],
  'arrival_times': [0, 0, 0],
  'departure_times': [5, 6, 10],
  'to': 'Olten',
  'departure_probabilities': [(5, 0.3333333333333333),
   (6, 0.3333333333333333),
   (10, 0.3333333333333333)],
  'arrival_probabilities': [(0, 1.0), (0, 1.0), (0, 1.0)]},
 'Olten': {'arr_dep_tuples': [(15, 20), (15, 20), (20, 25)],
  'arrival_times': [15, 15, 20],
  'departure_times': [20, 20, 25],
  'to': 'Bern',
  'departure_probabilities': [(20, 0.6666666666666666),
   (20, 0.6666666666666666),
   (25, 0.3333333333333333)],
  'arrival_probabilities': [(15, 0.6666666666666666),
   (15, 0.6666666666666666),
   (20, 1.0)]},
 'Bern': {'arr_dep_tuples': [(30, 35), (32, 37), (35, 40)],
  'arrival_times': [30, 32, 35],
  'departure_times': [35, 37, 40],
  'to': 'Brig',
  'departure_probabilities': [(35, 0.3333333333333333),
   (37, 0.3333333333333333),
   (40, 0.3333333333333333)],
  'arrival_probabilities': [(30, 0.3333333333333333),
   (32, 0.66666666

In [5]:
# Now - also for simplicity, compute the probability of making a connection (specifically, from Zurich to Olten)

In [6]:
def compute_connection_probability(
    departure_time_probabilities: List[int],
    arrival_time_probabilities: List[float],
    stations: List,
    station: str,
    transfer_time=5,
) -> float:
    # @todo -> refactor a bit (to make it useful for the next steps, e.g. how the parameters are passed and how the data is stored)
    """
    Compute the probability of making a connection given a list of departure times and a list of arrival time probabilities
    """
    # for each departure time, compute the probability of making a connection
    probabilities = []
    # multiply probability of departing at time t with the probability of arriving at or before time (t - transfer time)
    for i in range(len(departure_time_probabilities)):
        # only consider the arrival time probabilities that are before the departure time
        # @todo -> solve this with a precomputed list of probabilities (since we need it more often)
        # @todo -> maybe use combined (multiplied) probability that a certain train departs at time t' when it arrives at or before time t
        arrival_probability = compute_probability_to_arrive_at_or_before(
            departure_time_probabilities[i][0] - transfer_time,
            stations[station]["arrival_times"],
        )
        probabilities.append(departure_time_probabilities[i][1] * arrival_probability)
    # sum the probabilities
    probability = sum(probabilities)
    return probability


# we assume that we already have unique lists of tuples representing the departure and arrival times and their probabilities
# sort the lists by time
departure_probabilities = list(set(stations["Olten"]["departure_probabilities"]))
departure_probabilities.sort(key=lambda x: x[0])
arrival_probabilities = list(set(stations["Olten"]["arrival_probabilities"]))
arrival_probabilities.sort(key=lambda x: x[0])
made_connection_1_2 = compute_connection_probability(
    departure_probabilities, arrival_probabilities, stations, "Olten"
)

In [13]:
# Now compute probability that trip 2 arrives at the third station given that we make the connection between the first and second trip (=between first and second station)

t = 30
departure_probabilities = list(set(stations["Olten"]["departure_probabilities"]))
departure_probabilities.sort(key=lambda x: x[0])
arrival_probabilities_second_stop = list(set(stations["Bern"]["arrival_probabilities"]))
arrival_probabilities_second_stop.sort(key=lambda x: x[0])
arrival_times_first_stop = stations["Olten"]["arrival_times"]
arrival_probabilities_first_stop = list(set(stations["Olten"]["arrival_probabilities"]))
arrival_probabilities_first_stop.sort(
    key=lambda x: x[0]
)  # @todo ordering here correct?

print("Departure probabilities: ", departure_probabilities)
arrival_departure_tuples_second_station = [(20, 30), (20, 32), (25, 35)]


def compute_probability_to_arrive_at_t_given_made_connection(
    time: int,
    departure_probabilities: List,
    arrival_probabilities_second_stop: List,
    arrival_times_first_stop: List[int],
    probability_connection_made: float,
    arrival_departure_tuples_second_station: List,
    transfer_time=5,
) -> float:
    """Compute the probability of arriving at time t given that we made the connection."""
    probabilities = []
    # loop through all possible departure times
    for t_dep in departure_probabilities:
        # probability of arriving at time t given the departure at t'
        # we need to count how often an arrival-departure pair occurs (divided by total number of arrival-departure pairs) and then divide by the probability that we depart at t'
        occurence_arrival_departure_pair = 0
        # arrival times included in the arrival probabilities
        # @todo -> here we need the tuples!, count otherwise wrong!, refactor please :)
        for pair in arrival_departure_tuples_second_station:
            if t_dep[0] == pair[0] and time == pair[1]:
                occurence_arrival_departure_pair += 1
        probability_arrival_t_departure_t_prime = (
            occurence_arrival_departure_pair / len(arrival_probabilities_second_stop)
        )
        probability_arrival_given_departure = (
            probability_arrival_t_departure_t_prime / t_dep[1]
        )
        print("Departure time: ", t_dep[0])
        print("occurence_arrival_departure", occurence_arrival_departure_pair)
        # print("probability given departure", probability_arrival_given_departure, "occurence_arrival_departure", occurence_arrival_departure_pair)
        # probability that we depart at t'
        probability_departure = t_dep[1]
        # probability that we arrive before t' - transfer time
        probability_arrival_before_t_prime = compute_probability_to_arrive_at_or_before(
            t_dep[0] - transfer_time, arrival_times_first_stop
        )
        print("Probability arrival before t': ", probability_arrival_before_t_prime)
        # probability that we arrive at t given that we made the connection
        print(
            "Multiply",
            probability_departure,
            probability_arrival_before_t_prime,
            probability_arrival_given_departure,
            "divide by",
            probability_connection_made,
        )
        probability = (
            probability_departure
            * probability_arrival_before_t_prime
            * probability_arrival_given_departure
        ) / probability_connection_made
        probabilities.append(probability)
    probability_sum = sum(probabilities)
    return probability_sum


compute_probability_to_arrive_at_t_given_made_connection(
    t,
    departure_probabilities,
    arrival_probabilities_second_stop,
    arrival_times_first_stop,
    made_connection_1_2,
    arrival_departure_tuples_second_station,
)

Departure probabilities:  [(20, 0.6666666666666666), (25, 0.3333333333333333)]
Departure time:  20
occurence_arrival_departure 1
time limit: 15
arrival times: [15, 15, 20]
max arrival time: 15
probabilities: [0.6666666666666666]
Probability arrival before t':  0.6666666666666666
Multiply 0.6666666666666666 0.6666666666666666 0.5 divide by 0.7777777777777777
Departure time:  25
occurence_arrival_departure 0
time limit: 20
arrival times: [15, 15, 20]
max arrival time: 20
probabilities: [0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333]
Probability arrival before t':  1.0
Multiply 0.3333333333333333 1.0 0.0 divide by 0.7777777777777777


0.28571428571428575